#Problem Explanation

*  First what is load balancing algorithm ?
  It is a set of rules that a load balancer follows to determine the best server for each of the different client requests. (AWS)

  * So here we are trying to write down the "set of rules" using python, and we need to test it at least on 2 devices, since obviously we are trying to distribute the request across two or multiple servers (devices)

In [1]:
# importing the dependencies
import tensorflow as tf  # To simulate heavy computational tasks (Matrix multiplication)
import numpy as np  # math purposes
import time # In our case (Measuring execution time: Track how long a code block takes to run)


In [2]:
# Fucntion to simulate heavy computation --> Matrix multiplication
def compute_on_device(device_name):

  with tf.device(device_name):


      start_time = time.time()

      # Create a latge random matrix
      matrix_size = 1000
      a = tf.random.normal([matrix_size, matrix_size])
      b = tf.random.normal([matrix_size, matrix_size])

      result = tf.matmul(a, b)
      end_time = time.time()

      return (end_time - start_time)

# Least Connections

In [12]:
# here i assumed that the cpu has two connections
# so when i called the function the tasks were assigned to the gpu since it has the least connections
# after a couple of tasks the gpu will have more connection thus, the cpu will be assigned more tasks
# note that in this code there is no maximum connections specified for each device, because i wanted to foucus on the main concept of the algorithm

devices = {
    "/CPU:0": 2,  # Current connections
    "/GPU:1": 0,
}



def least_connections(devices, num_tasks):
  for i in range(num_tasks):

    # find the device with the least ammount connections
    # minimum "value" is our key
    device = min(devices, key=devices.get)
    # conpute on the chosen device from the previous code of line
    task_time = compute_on_device(device)

    # updation statement +1 connections each time a task is "computed"
    devices[device] += 1 # remeber its a dict
    print(f"Task {i + 1} executed on {device}, Time: {task_time:.4f} seconds")


# call the fucntion
least_connections(devices, 10)


Task 1 executed on /GPU:1, Time: 0.0735 seconds
Task 2 executed on /GPU:1, Time: 0.0718 seconds
Task 3 executed on /CPU:0, Time: 0.0664 seconds
Task 4 executed on /GPU:1, Time: 0.0684 seconds
Task 5 executed on /CPU:0, Time: 0.0687 seconds
Task 6 executed on /GPU:1, Time: 0.0755 seconds
Task 7 executed on /CPU:0, Time: 0.0757 seconds
Task 8 executed on /GPU:1, Time: 0.0695 seconds
Task 9 executed on /CPU:0, Time: 0.0722 seconds
Task 10 executed on /GPU:1, Time: 0.0721 seconds


# Weighted Round robin

In [15]:
devices = {"/CPU:0": 3, "/GPU:1": 4} # here we used the dict data structure in order to "link" the device with its weight

def weighted_round_robin(devices, num_tasks):

    # this code snippet is used to create a lists where each device appears multiple times == its weight
    device_list = []
    for device, weight in devices.items():
        device_list.extend([device] * weight)

    # Loop to distribute tasks, here the tasks will be distributed first to the cpu since its first in the line
    for i in range(num_tasks):
        device = device_list[i % len(device_list)]  # Select device based on the current index

        # Simulate task computation on the selected device
        task_time = compute_on_device(device)
        print(f"Task {i + 1} executed on {device}, Time: {task_time:.4f} seconds")

# Calling the fucntion
weighted_round_robin(devices, 10)

Task 1 executed on /CPU:0, Time: 0.0719 seconds
Task 2 executed on /CPU:0, Time: 0.0679 seconds
Task 3 executed on /CPU:0, Time: 0.0696 seconds
Task 4 executed on /GPU:1, Time: 0.0728 seconds
Task 5 executed on /GPU:1, Time: 0.0712 seconds
Task 6 executed on /GPU:1, Time: 0.0666 seconds
Task 7 executed on /GPU:1, Time: 0.0721 seconds
Task 8 executed on /CPU:0, Time: 0.0688 seconds
Task 9 executed on /CPU:0, Time: 0.0667 seconds
Task 10 executed on /CPU:0, Time: 0.0717 seconds


# IP Hash

In [14]:
# This algorithm is concered with privacy and security as it appers for its name
# here we want to map the task to the server based on the ip address of the client that requested the task
devices  = ["/CPU:0", "/GPU:1", "/CPU:1"]

import hashlib

# this fucntion is to hash the ip address using md5 encryption
def ip_hash(client_ip, servers):

  # calculate the hash of the clients ip
  hash_value = int(hashlib.md5(client_ip.encode()).hexdigest(), 16)
  # this line servers to determine which server to assign a request based on the hash value that is calculated
  # (hash-based selection)
  server_index = hash_value % len(servers)

  return servers[server_index] # here is the selection

def process_requests(client_ips):
  # note: enumerate(list) generates pairs of (index, item) for each item in the  list.
  for i, client_ip in enumerate(client_ips): # here the i will iterate for the index and the client_ip will iterate for the item :), very nice actually

    server = ip_hash(client_ip, devices)

    task_time = compute_on_device(server)
    print(f"Request {i + 1} from {client_ip} executed on {server}, Time: {task_time:.4f} seconds")


# the IPs where generated using chat gpt
client_ips = ["192.168.1.10", "192.168.1.11", "192.168.1.10", "192.168.1.12", "192.168.1.11"]
process_requests(client_ips)





Request 1 from 192.168.1.10 executed on /GPU:1, Time: 0.0700 seconds
Request 2 from 192.168.1.11 executed on /GPU:1, Time: 0.0697 seconds
Request 3 from 192.168.1.10 executed on /GPU:1, Time: 0.0671 seconds
Request 4 from 192.168.1.12 executed on /CPU:1, Time: 0.0690 seconds
Request 5 from 192.168.1.11 executed on /GPU:1, Time: 0.0737 seconds


# Report


In this notebook i chose to work on the following load balancing algorithms: Least connections, Weighted Round Robin, IP has.
# Brief explanation for each algorithm
1. The least connections
is a simple algorithm that tracks the number of connections a server has with a client and  chooses the server to use by prioritizing the device with the least connections as its name implies, to check the results [Least Connections](https://colab.research.google.com/drive/1XPbVQ0UNutsKc49p-kWI24q6SLDC2PLO#scrollTo=IW8M7SyuJ9lv&line=1&uniqifier=1)

2. Weighted Round Robin
 The algorithm is based on the assigned weights for each device which is the maximum load allowed for each device and to check the results  [Weighted Round Robin](https://colab.research.google.com/drive/1XPbVQ0UNutsKc49p-kWI24q6SLDC2PLO#scrollTo=6hc7aaTMIoSL&line=1&uniqifier=1)

 3. IP hash
 as the name implies this algorithm is concerned with privacy and encryption in addition to its main purpose (tasks distribution), here the tasks are mapped to each server based on the ip address of the client that requested the task, and the ip address is hashed, you can check the results  [IP Hash](https://colab.research.google.com/drive/1XPbVQ0UNutsKc49p-kWI24q6SLDC2PLO#scrollTo=toj0d8ZCPGHe&line=1&uniqifier=1)

Finally, Comparison of performance
1. least connections --> based on the results; responsiveness and efficiency but may lead to uneven load if one device quickly fills up.
gpu on average --> 0.0723 seconds
cpu pn average --> 0.0714 seconds
execution time --> gpu > cpu

2. Weighted Round Robin  --> More consistent than the Least Connections method, but the CPU can become a bottleneck with increased load.
gpu on average --> 0.0707 seconds
cpu pn average --> 0.0696 seconds
execution time --> gpu > cpu

3. IP hash --> based on the  distribution of client IPs; might lead to inefficiencies if certain servers are overwhelmed by repeated requests from a small number of clients.
gpu on average --> 0.0701 seconds
cpu pn average -->0.0690 seconds only one task was performed on the cpu
execution time --> gpu > cpu

surprisingly the cpu performs faster gpu in our case
I think the reason behind that is the gpu where assigned more tasks so the average execution time was higher


